In [1]:
import os
import pandas as pd
import nltk
nltk.download('wordnet')
wn = nltk.corpus.wordnet
from digital_manuscript import BnF

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/matthewkumar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
path = os.getcwd() + '/../m-k-manuscript-data/manuscript-object/thesaurus/animal.csv'

df = pd.read_csv(path)
terms = list(set(df['prefLabel_en']))
simple_terms = [t for t in terms if ' ' not in t]
simple_terms

['mutton',
 'hog',
 'snake',
 'dogfish',
 'swan',
 'oyster',
 'weasel',
 'goat',
 'dormouse',
 'horse',
 'nightingale',
 'lamb',
 'sheep',
 'duck',
 'chaffinch',
 'butterfly',
 'siskin',
 'mealworm',
 'animal',
 'grasshopper',
 'mussel',
 'aucupio',
 'colt',
 'hart',
 'wolf',
 'shell',
 'rabbit',
 'barbel',
 'turtle',
 'water-dog',
 'pork',
 'flea',
 'toad',
 'mule',
 'fish',
 'bat',
 'codfish',
 'tellin',
 'swine',
 'fly',
 'hen',
 'partridge',
 'worm',
 'dog',
 'silkworm',
 'cow',
 'lark',
 'turtledofe',
 'crow',
 'og',
 'goldfinch',
 'chick',
 'cat',
 'louse',
 'petit-gri',
 'rat',
 'frog',
 'calandra',
 'passerine',
 'earthworm',
 'human',
 'ortolan',
 'verdaule',
 'linnet',
 'ant',
 'calendra',
 'crayfish',
 'dragon',
 'calf',
 'bird',
 'pig',
 'cicada',
 'chicken',
 'cuttlefish',
 'ox',
 'beef',
 'pigeon',
 'spider',
 'ewe',
 'eagle',
 'bombicum',
 'mouse',
 'swallow',
 'viper',
 'porcupine',
 'bittern',
 'snail',
 'squirrel',
 'crab',
 'cuckoo',
 'lizard',
 'daot']

In [3]:
term = 'dog'
dog = wn.synsets(term, pos=wn.NOUN)
dog
# hyper = dog[0].hypernyms()
# for x in hyper:
#     print(x, x.hypernyms())

# from nltk.corpus import wordnet as wn
# for ss in wn.synsets(term):
#     print (ss, ss.hypernyms())

[Synset('dog.n.01'),
 Synset('frump.n.01'),
 Synset('dog.n.03'),
 Synset('cad.n.01'),
 Synset('frank.n.02'),
 Synset('pawl.n.01'),
 Synset('andiron.n.01')]

In [4]:
animal = wn.synset('animal.n.01')

class r_node:
    def __init__(self, lemma, children=[]):
        self.lemma = lemma
        
#         if lemma == animal:
#             print(children + [lemma,])
        
        self.children = children
        self.hypernyms = lemma.hypernyms()
        self.parents = [r_node(h, children + [lemma,]) for h in self.hypernyms]
        
    def return_animal_path(self):
        if self.lemma == animal:
            return [[self.lemma,],]
        else:
            parent_paths = [n.return_animal_path() for n in self.parents]
            parent_paths = [p for p in parent_paths if len(p) > 0]
            paths = []
            for p in parent_paths:
                paths += p
            for p in paths:
                p.append(self.lemma)
            return paths

In [5]:
dog = wn.synsets('dog', pos=wn.NOUN)[0]
dn = r_node(dog)
dn.return_animal_path()
# print([p.parents[0].lemma for p in dn.parents])

# animal


[[Synset('animal.n.01'),
  Synset('chordate.n.01'),
  Synset('vertebrate.n.01'),
  Synset('mammal.n.01'),
  Synset('placental.n.01'),
  Synset('carnivore.n.01'),
  Synset('canine.n.02'),
  Synset('dog.n.01')],
 [Synset('animal.n.01'), Synset('domestic_animal.n.01'), Synset('dog.n.01')]]

In [6]:
squirrel = wn.synsets('squirrel')[0]
sn = r_node(squirrel)
sn.return_animal_path()

[[Synset('animal.n.01'),
  Synset('chordate.n.01'),
  Synset('vertebrate.n.01'),
  Synset('mammal.n.01'),
  Synset('placental.n.01'),
  Synset('rodent.n.01'),
  Synset('squirrel.n.01')]]

In [7]:
fish = wn.synsets('fish', pos=wn.NOUN)[0]
fn = r_node(fish)
fn.return_animal_path()

[[Synset('animal.n.01'),
  Synset('chordate.n.01'),
  Synset('vertebrate.n.01'),
  Synset('aquatic_vertebrate.n.01'),
  Synset('fish.n.01')]]

In [8]:
louse = wn.synsets('louse')[0]
ln = r_node(louse)
ln.return_animal_path()

[[Synset('animal.n.01'),
  Synset('invertebrate.n.01'),
  Synset('arthropod.n.01'),
  Synset('insect.n.01'),
  Synset('louse.n.01')]]

In [9]:
oyster = wn.synsets('oyster')[0]
on = r_node(oyster)
on.return_animal_path()

[[Synset('animal.n.01'),
  Synset('invertebrate.n.01'),
  Synset('mollusk.n.01'),
  Synset('bivalve.n.01'),
  Synset('oyster.n.01')]]

In [10]:
def get_animal_paths(lemma):
    paths = []
    node = r_node(lemma)
    
    
    
    

In [11]:
squirrel = wn.synsets('squirrel')[0]
weasel = wn.synsets('weasel')[1] # index 0 is a sneaky person
squirrel.lowest_common_hypernyms(weasel)

snake = wn.synsets('snake')[0]
print(snake.lowest_common_hypernyms(weasel))
print(snake.lowest_common_hypernyms(squirrel))

turtle = wn.synsets('turtle')[1] # index 0 is a turtleneck (lmao)
print(turtle.lowest_common_hypernyms(snake))
print(turtle.lowest_common_hypernyms(squirrel))

fish = wn.synsets('fish', pos=wn.NOUN)[0] # pos = part of speech
swan = wn.synsets('swan')[0]
print(fish.lowest_common_hypernyms(turtle))
print(fish.lowest_common_hypernyms(snake))

louse = wn.synsets('louse')[0]
oyster = wn.synsets('oyster')[0]
print(louse.lowest_common_hypernyms(snake))
print(louse.lowest_common_hypernyms(oyster))

[Synset('vertebrate.n.01')]
[Synset('vertebrate.n.01')]
[Synset('reptile.n.01')]
[Synset('vertebrate.n.01')]
[Synset('vertebrate.n.01')]
[Synset('vertebrate.n.01')]
[Synset('animal.n.01')]
[Synset('invertebrate.n.01')]
